<a href="https://colab.research.google.com/github/PedroSilvah/alura-google_projeto/blob/main/imers%C3%A3o_alura%2Bgoogle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import requests

# --- Funções da API ---
def buscar_dados_api(endpoint, headers=None):
    """
    Realiza uma requisição GET à API e retorna os dados em formato JSON.
    Args:
        endpoint (str): URL completa do endpoint da API.
        headers (dict, optional): Dicionário contendo os cabeçalhos da requisição,
                                    como tokens de autenticação.
    Returns:
        dict: Dados recebidos da API em formato JSON, se a requisição for bem-sucedida.
              Retorna None em caso de erro na requisição.
    """
    try:
        resposta = requests.get(endpoint, headers=headers)
        resposta.raise_for_status()  # Verifica se a requisição foi bem-sucedida
        return resposta.json()  # Converte a resposta da API para formato JSON
    except requests.exceptions.RequestException as e:
        print(f"Erro ao acessar a API: {e}")
        return None

# --- Funções de Análise ---
def analisar_planilha(planilha, api_endpoint=None, api_headers=None):
    """
    Analisa uma planilha de despesas e receitas, identifica anormalidades
    e integra dados de uma API, se fornecida.
    Args:
        planilha (str): Nome do arquivo da planilha em formato CSV.
        api_endpoint (str, optional): URL completa do endpoint da API.
        api_headers (dict, optional): Dicionário contendo os cabeçalhos da requisição da API.
    Returns:
        tuple: Um DataFrame Pandas com a planilha analisada e um dicionário
                contendo as informações da análise.
    """
    df = pd.read_csv(planilha, sep=",", header=0, index_col=0)

    # Convertendo colunas de valores para tipo numérico
    for col in df.columns:
        df[col] = df[col].str.replace('R\$ ', '', regex=False).str.replace('.', '', regex=False).str.replace(',', '.', regex=False).astype(float)

    # Cálculo das DESPESAS TOTAIS
    df.loc['DESPESAS TOTAIS'] = df.iloc[0:7].sum()

    # Cálculo do LUCRO MENSAL (corrigido)
    df.loc['LUCRO MENSAL'] = df.loc['RENDIMENTO BRUTO'] - df.loc['DESPESAS TOTAIS']

    # Análise de Anormalidades
    analise = {
        'operacoes_corretas': True,  # Assume que as operações estão corretas inicialmente
        'anormalidades': {},
        'possiveis_causas': {}
    }

    # Analisando DESPESAS REPOSIÇÕES: Verifica valores fora do desvio padrão
    reposicoes = df.loc['DESPESAS REPOSIÇÕES']
    media_reposicoes = reposicoes.mean()
    desvio_padrao_reposicoes = reposicoes.std()
    anomalias_reposicoes = reposicoes[(reposicoes > media_reposicoes + desvio_padrao_reposicoes) |
                                     (reposicoes < media_reposicoes - desvio_padrao_reposicoes)]
    if not anomalias_reposicoes.empty:
        analise['anormalidades']['DESPESAS REPOSIÇÕES'] = anomalias_reposicoes.to_dict()
        analise['possiveis_causas']['DESPESAS REPOSIÇÕES'] = [
            "Sazonalidade: Aumento na demanda em determinados períodos do ano.",
            "Entrada de novos produtos: Lançamento de novos produtos ou aumento na variedade do estoque.",
            "Compras em grande quantidade: Aproveitamento de ofertas ou descontos.",
            "Erro de digitação: Conferir os valores para verificar se houve algum erro na digitação."
        ]

    # Analisando DESPESAS IMPREVISTAS: Verifica valores muito acima do desvio padrão
    imprevistas = df.loc['DESPESAS IMPREVISTAS']
    media_imprevistas = imprevistas.mean()
    desvio_padrao_imprevistas = imprevistas.std()
    anomalias_imprevistas = imprevistas[(imprevistas > media_imprevistas + (2 * desvio_padrao_imprevistas)) |
                                      (imprevistas < media_imprevistas - (2 * desvio_padrao_imprevistas))]
    if not anomalias_imprevistas.empty:
        analise['anormalidades']['DESPESAS IMPREVISTAS'] = anomalias_imprevistas.to_dict()
        analise['possiveis_causas']['DESPESAS IMPREVISTAS'] = [
            "Despesas específicas: Analisar a natureza de cada despesa para entender a oscilação.",
            "Sazonalidade: Alguns tipos de despesas imprevistas podem ser sazonais.",
            "Erro de lançamento: Verificar se houve erro na digitação ou lançamento da informação."
        ]

    # Analisando DESPESA COMBUSTÍVEL: Verifica variações percentuais muito grandes
    combustivel = df.loc['DESPESA COMBUSTÍVEL']
    variacao_percentual = combustivel.pct_change()
    anomalias_combustivel = variacao_percentual[abs(variacao_percentual) > 0.15]
    if not anomalias_combustivel.empty:
        analise['anormalidades']['DESPESA COMBUSTÍVEL'] = anomalias_combustivel.to_dict()
        analise['possiveis_causas']['DESPESA COMBUSTÍVEL'] = [
            "Variação no preço do combustível: O aumento do preço dos combustíveis pode explicar a tendência de alta.",
            "Mudança na demanda: Aumento na demanda por transporte ou logística da empresa.",
            "Eficiência da frota: Possível queda na eficiência da frota, demandando maior consumo de combustível."
        ]

    # Exemplo de uso da API (substitua pela lógica específica da sua API)
    if api_endpoint:
        dados_api = buscar_dados_api(api_endpoint, headers=api_headers)
        if dados_api:
            # Integre os dados da API à análise
            if 'inflacao_mensal' in dados_api:  # Adapte para o nome do dado na sua API
                df.loc['Inflação'] = dados_api['inflacao_mensal']

    return df, analise

def comparar_planilhas(df1, df2):
    """
    Compara duas planilhas e retorna um resumo das diferenças nos valores anuais.
    Args:
        df1 (pd.DataFrame): DataFrame da primeira planilha.
        df2 (pd.DataFrame): DataFrame da segunda planilha.
    Returns:
        dict: Dicionário contendo as diferenças percentuais entre as planilhas
              para cada item analisado.
    """
    comparacao = {}
    for item in ['DESPESAS REPOSIÇÕES', 'DESPESAS FUNCIONARIOS',
                 'DESPESAS ALUGUEL', 'DESPESA COMBUSTÍVEL',
                 'DESPESAS ESCRITÓRIO', 'DESPESA ENERGIA ELÉTRICA',
                 'RENDIMENTO BRUTO']:
        valor_2022 = df1.loc[item, 'TOTAL ANUAL']
        valor_2023 = df2.loc[item, 'TOTAL ANUAL']
        variacao = ((valor_2023 - valor_2022) / valor_2022) * 100
        comparacao[item] = f"Aumento de {variacao:.1f}%"
    return comparacao

# --- Loop Principal ---
planilhas = []
while True:
    arquivo = input("Digite o nome do arquivo da planilha (ou 'sair' para finalizar): ")
    if arquivo.lower() == 'sair':
        break
    api_endpoint = input("Digite a URL da API (opcional): ")  # Permite ao usuário fornecer a URL da API
    api_headers = {}  # Crie um dicionário para os headers da API, se necessário
    df, analise = analisar_planilha(arquivo, api_endpoint, api_headers)  # Chama a função de análise
    planilhas.append(df)
    print("\n--- Análise da Planilha ---")
    print(f"Operações Matemáticas Corretas: {analise['operacoes_corretas']}")
    if analise['anormalidades']:
        print("\nAnormalidades Encontradas:")
        for categoria, valores in analise['anormalidades'].items():
            print(f"- {categoria}:")
            for mes, valor in valores.items():
                print(f"   * {mes}: R$ {valor:,.2f}")
        print("\nPossíveis Causas:")
        for categoria, causas in analise['possiveis_causas'].items():
            print(f"- {categoria}:")
            for i, causa in enumerate(causas):
                print(f"   {i+1}. {causa}")
    if len(planilhas) == 2:  # Compara as planilhas quando duas são fornecidas
        comparacao = comparar_planilhas(planilhas[0], planilhas[1])
        print("\n--- Comparação Entre as Planilhas ---")
        for item, variacao in comparacao.items():
            print(f"* {item}: {variacao}")
        planilhas = []  # Reinicia a lista para próximas comparações